In [1]:
import random
import numpy as np
import tensorflow as tf
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6120)])
#   except RuntimeError as e:
#     print(e)

In [3]:
lemmatizer=WordNetLemmatizer()
data = open("intense_spanish.json").read()
intents = json.loads(data)
words = pickle.load(open('words_spanish.pkl','rb'))
clases = pickle.load(open('clases_spanish.pkl','rb'))
model = tf.keras.models.load_model('Chatbot_base_spanish.tf')

In [4]:
model.summary()

Model: "chatbot_trainer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  3456      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  8256      
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-t

In [5]:
# model.predict([])

In [6]:
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)
                      for word in sentence_words]
    return sentence_words



In [7]:
# this function will return 1 if the word is contained in our sentence and in the list previously defined 
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [8]:
# This function predicts a cleansed data with a mark in the bow (flag = 1 )
# It also appends the class - tag in the clases list where it belongs 
def predict_clases(sentence,model):
    print(clean_up_sentences(sentence))
    p = bow(sentence,words,show_details=True)
    
    res = model.predict(np.array([p]))[0]
    
    ERROR_THRESHOLD=0.35#0.25
    results = [[ i , r ] for i , r  in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key = lambda x : x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({
            'intent' : clases[r[0]],
            'probability': str(r[1])

        } )
        return return_list


In [9]:
# this function will print a random response in the class according the input that the user made
def get_response(intents_list,intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json ['intents']
    result = ""
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print ("Esta vivo!")

Esta vivo!


In [10]:


# while True:
#     message = ""
#     ints = predict_clases(message)
#     res = get_response(ints , intents)
#     print(res)

#     # if message == 'salir':
    #     C=False


In [11]:
# message = "whats your name?"

# ints = predict_clases(message,model)
# res = get_response(ints , intents)
# print(res)

In [17]:
message = "hola buenas tardes , mucho gusto mi nombre es maria , quien eres tu  ?"

ints = predict_clases(message,model)
res = get_response(ints , intents)
print(res)

['hola', 'buenas', 'tardes', ',', 'mucho', 'gusto', 'mi', 'nombre', 'e', 'maria', ',', 'quien', 'eres', 'tu', '?']
found in bag: hola
found in bag: buenas
found in bag: nombre
found in bag: e
found in bag: tu
1/1 [==============================] - 0s 12ms/step
Llámame FINANCIALYTICS-BOT,soy bot de apoyo en Financialitica
